In [ ]:
# function
import os
import sys

new_directory = "E:/airflow/airflow"
current_directory = os.getcwd()
scripts_path = os.path.abspath(os.path.join(os.getcwd(), '../scripts'))

def change_directory(current_directory, new_directory,scripts_path):
    # Get the current working directory
    print(f'Current directory: {current_directory}')
    # Define the path to change to
    new_directory = "E:/airflow/airflow"
    try:
        # Change the current working directory
        os.chdir(new_directory)
        # Verify the change
        current_directory = os.getcwd()
        print(f'Current directory changed to: {current_directory}')
    except FileNotFoundError:
        print(f'Error: The directory "{new_directory}" does not exist.')
    except PermissionError:
        print(f'Error: Permission denied to change to "{new_directory}".')
    except Exception as e:
        print(f'An unexpected error occurred: {e}')
    # Add the scripts directory to the Python path
    sys.path.append(scripts_path)

    
change_directory(current_directory, new_directory, scripts_path)

In [ ]:
# Setting up all directory
root_folder = "E:/airflow/airflow"
database_path = root_folder+"/database/"
data_directory = root_folder+"/data/raw/"
data_profile_path = root_folder+"/data/profile_report/"
intermediate_data_path = root_folder+"/data/interim/"
final_processed_data_path = root_folder+"/data/processed/"

old_data_directory = root_folder+"/data/raw/"
new_data_directory = root_folder+"/data/new/"
intermediate_path = root_folder+"/data/interim/"


# Database
db_path = root_folder+"/database/"
db_file_name = "feature_store_v01.db"
drfit_db_name = "drift_db_name.db"
date_columns = ['registration_init_time','transaction_date_min','transaction_date_max','membership_expire_date_max','last_login']
drift_db_name = "drift_db_name.db"

# Mlflow
mlflow_tracking_uri = "http://Localhost:6006"
ml_flow_model_path = root_folder+ "/mlruns/2/cb66e22bcbf74ded99dc219eb29e7609/artifacts/models/"
ml_flow_path = root_folder+ "/mlruns/2/cb66e22bcbf74ded99dc219eb29e7609"

run_on = "old" #"old"
append=False
date_transformation = False
start_date = '2017-03-01'
end_date = '2017-03-31'

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
%matplotlib inline
from scripts.utils import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from datetime import datetime
from ydata_profiling import ProfileReport

In [ ]:
%%time
members_n, user_logs_n, transactions_n, train_n  = load_data( [
                                                            f"{new_data_directory}members_profile_new.csv",
                                                            f"{new_data_directory}user_logs_new.csv",
                                                            f"{new_data_directory}transactions_logs_new.csv",
                                                            f"{new_data_directory}churn_logs_new.csv"
                                                            ]
                                                          )

In [ ]:
print(members_n.shape)
print(transactions_n.shape)
print(user_logs_n.shape)
print(train_n.shape)

In [ ]:
user_logs_n['date'] = utils.fix_time_in_df(user_logs_n, 'date', expand=False)

In [ ]:
%%time
members, user_logs, transactions, train  = utils.load_data( [
                                                            f"{data_directory}members_profile.csv",
                                                            f"{data_directory}userlogs.csv",
                                                            f"{data_directory}transactions_logs.csv",
                                                            f"{data_directory}churn_logs.csv"
                                                            ]
                                                          )

In [ ]:
print(members.shape)
print(transactions.shape)
print(user_logs.shape)
print(train.shape)

In [ ]:
user_logs_n.head()

In [ ]:
members_list = np.unique(list(members['msno']))

In [ ]:
train_members_list = np.unique(list(train['msno']))

In [ ]:
user_logs_n[(user_logs_n['date']>'2017-03-01') & 
            (user_logs_n['date']<'2017-03-31') &
            (user_logs_n['msno'].isin(members_list)) & 
            (user_logs_n['msno'].isin(train_members_list))].shape

In [ ]:
user_logs_n[(user_logs_n['date']>'2017-03-01') & 
            (user_logs_n['date']<'2017-03-31') &
            (user_logs_n['msno'].isin(members_list)) & 
            (user_logs_n['msno'].isin(train_members_list))].to_csv("/home/data/new_append/user_logs_march.csv",index=False)

In [ ]:
%%time
transactions_n['transaction_date'] = utils.fix_time_in_df(transactions_n, 'transaction_date', expand=False)
transactions_n['membership_expire_date'] = utils.fix_time_in_df(transactions_n, 'membership_expire_date', expand=False)

In [ ]:
transactions_n.head()

In [ ]:
transactions_n[(transactions_n['transaction_date']>'2017-03-01') & (transactions_n['transaction_date']<'2017-03-31') & (transactions_n['membership_expire_date']<'2017-12-31')].to_csv("/home/data/new_append/transactions_logs_march.csv",index=False)

In [ ]:
transactions_n[(transactions_n['transaction_date']>'2017-03-01') & 
               (transactions_n['transaction_date']<'2017-03-31') & 
               (transactions_n['membership_expire_date']<'2017-12-31') & 
               (transactions_n['msno'].isin(members_list)) & 
               (transactions_n['msno'].isin(train_members_list))].shape

In [ ]:
transactions_n[(transactions_n['transaction_date']>'2017-03-01') & 
               (transactions_n['transaction_date']<'2017-03-31') & 
               (transactions_n['membership_expire_date']<'2017-12-31') & 
               (transactions_n['msno'].isin(members_list)) & 
               (transactions_n['msno'].isin(train_members_list))].to_csv("/home/data/new_append/transactions_logs_march.csv",index=False)

In [ ]:
march_user_logs = user_logs_n[(user_logs_n['date']>'2017-03-01') & 
            (user_logs_n['date']<'2017-03-31') &
            (user_logs_n['msno'].isin(members_list)) & 
            (user_logs_n['msno'].isin(train_members_list))]

In [ ]:
march_transactions = transactions_n[(transactions_n['transaction_date']>'2017-03-01') & 
               (transactions_n['transaction_date']<'2017-03-31') & 
               (transactions_n['membership_expire_date']<'2017-12-31') & 
               (transactions_n['msno'].isin(members_list)) & 
               (transactions_n['msno'].isin(train_members_list))]

In [ ]:
user_logs_updated = user_logs.append(march_user_logs)

In [ ]:
user_logs_updated.head()

In [ ]:
user_logs_updated.shape

In [ ]:
transactions_updated = transactions.append(march_transactions)
transactions_updated.shape

In [ ]:
def get_new_data_appended(new_data_directory, start_data, end_date):
    members_n, user_logs_n, transactions_n, train_n  = utils.load_data( [
                                                            f"{new_data_directory}members_profile_new.csv",
                                                            f"{new_data_directory}user_logs_new.csv",
                                                            f"{new_data_directory}transactions_logs_new.csv",
                                                            f"{new_data_directory}churn_logs_new.csv"
                                                            ]
                                                          )
    
    #get the list of memebers fron historical data. This assumes, no new user has been added in the system. Shouldn't be done, when new users are added
    members_list = np.unique(list(members['msno']))
    train_members_list = np.unique(list(train['msno']))
    
    #Some Date Filters are manual at this point for sanity check 
    user_logs_n['date'] = utils.fix_time_in_df(user_logs_n, 'date', expand=False)
    march_user_logs = user_logs_n[(user_logs_n['date']>start_data) & 
            (user_logs_n['date']<end_date) &
            (user_logs_n['msno'].isin(members_list)) & 
            (user_logs_n['msno'].isin(train_members_list))]
    
    transactions_n['transaction_date'] = utils.fix_time_in_df(transactions_n, 'transaction_date', expand=False)
    transactions_n['membership_expire_date'] = utils.fix_time_in_df(transactions_n, 'membership_expire_date', expand=False)
    march_transactions = transactions_n[(transactions_n['transaction_date']>start_data) & 
               (transactions_n['transaction_date']<end_date) & 
               (transactions_n['membership_expire_date']<'2017-12-31') & 
               (transactions_n['msno'].isin(members_list)) & 
               (transactions_n['msno'].isin(train_members_list))]

    
    return march_user_logs, march_transactions

In [ ]:
%%time
start_data = '2017-03-01'
end_date = '2017-03-31'
new_data_directory = root_folder+"data/new/"

a,b = get_new_data_appended(new_data_directory, start_data, end_date)

In [ ]:
members_n.head()

In [ ]:
def get_change(current, previous):
    if current == previous:
        return 0
    try:
        return (abs(current - previous) / previous) * 100.0
    except ZeroDivisionError:
        return float('inf')
    
def get_data_drift(current_data, old_data, column_list,exclude_list, metric='std'):
    std_deviation_percentage = []
    mean_deviation_percentage = []
    for eachCol in column_list:
        if metric == 'std' and eachCol not in exclude_list:
            std1 = current_data[eachCol].std()
            std2 = old_data[eachCol].std()
            std_deviation_percentage.append(get_change(std1, std2))
        elif metric =='mean'and eachCol not in exclude_list:
            mean1 = current_data[eachCol].mean()
            mean2 = old_data[eachCol].mean()
            mean_deviation_percentage.append(get_change(mean1, mean2))
    print(std_deviation_percentage,mean_deviation_percentage)
    return np.mean(std_deviation_percentage), np.mean(mean_deviation_percentage)

In [ ]:
column_list = list(user_logs.select_dtypes(include=['int','float']).columns)
print(column_list)
get_data_drift(user_logs_n, user_logs, column_list,exclude_list= ['date'], metric='std')

In [ ]:
column_list = list(user_logs.select_dtypes(include=['int','float']).columns)
print(column_list)
get_data_drift(user_logs_n, user_logs, column_list,exclude_list= ['date'], metric='mean')

In [ ]:
column_list = list(transactions.select_dtypes(include=['int','float']).columns)
print(column_list)
get_data_drift(transactions_n, transactions, column_list,exclude_list= ['transaction_date','membership_expire_date'], metric='std')

In [ ]:
column_list = list(transactions.select_dtypes(include=['int','float']).columns)
print(column_list)
get_data_drift(transactions_n, transactions, column_list,exclude_list= ['transaction_date','membership_expire_date'], metric='mean')

In [ ]:
def get_new_data_appended(new_data_directory, start_data, end_date):
    members_n, user_logs_n, transactions_n, train_n  = utils.load_data( [
                                                            f"{new_data_directory}members_profile_new.csv",
                                                            f"{new_data_directory}user_logs_new.csv",
                                                            f"{new_data_directory}transactions_logs_new.csv",
                                                            f"{new_data_directory}churn_logs_new.csv"
                                                            ]
                                                          )
    
    #get the list of memebers fron historical data. This assumes, no new user has been added in the system. Shouldn't be done, when new users are added
    members_list = np.unique(list(members['msno']))
    train_members_list = np.unique(list(train['msno']))
    
    #Some Date Filters are manual at this point for sanity check 
    user_logs_n['date'] = utils.fix_time_in_df(user_logs_n, 'date', expand=False)
    march_user_logs = user_logs_n[(user_logs_n['date']>start_data) & 
            (user_logs_n['date']<end_date) &
            (user_logs_n['msno'].isin(members_list)) & 
            (user_logs_n['msno'].isin(train_members_list))]
    
    transactions_n['transaction_date'] = utils.fix_time_in_df(transactions_n, 'transaction_date', expand=False)
    transactions_n['membership_expire_date'] = utils.fix_time_in_df(transactions_n, 'membership_expire_date', expand=False)
    march_transactions = transactions_n[(transactions_n['transaction_date']>start_data) & 
               (transactions_n['transaction_date']<end_date) & 
               (transactions_n['membership_expire_date']<'2017-12-31') & 
               (transactions_n['msno'].isin(members_list)) & 
               (transactions_n['msno'].isin(train_members_list))]

    
    return march_user_logs, march_transactions

In [ ]:
# Declare Default arguments for the DAG
default_args = {
    'owner': 'CharlieThomas',
    'depends_on_past': False,
    'start_date': days_ago(2),
    'provide_context': True
}


# creating a new dag
dag = DAG('Model_Building_Pipeline', default_args=default_args, schedule_interval='0 0 * * 2', max_active_runs=1,tags=['ml_pipeline'])

# Integrating different operatortasks in airflow dag

op_reset_processes_flags = PythonOperator(task_id='reset_processes_flag',
                                         python_callable=get_flush_db_process_flags,
                                         op_kwargs={'db_path': db_path,'drfit_db_name':drfit_db_name},
                                         dag=dag)


op_create_db = PythonOperator(task_id='create_check_db', 
                            python_callable=build_dbs,
                            op_kwargs={'db_path': db_path, 'db_file_name': db_file_name},
                            dag=dag)


op_load_data = PythonOperator(task_id='load_data', 
                                python_callable=load_data_from_source,
                                  op_kwargs={'db_path': db_path, 'db_file_name': db_file_name,
                                             'drfit_db_name':drfit_db_name,
                                             'old_data_directory':old_data_directory,
                                             'new_data_directory':new_data_directory,
                                            'run_on':run_on,
                                              'start_date':start_date,
                                             'end_date':end_date},
                              dag=dag)
    

op_process_transactions = PythonOperator(task_id='process_transactions',
                                         python_callable=get_membership_data_transform,
                                         op_kwargs={'db_path': db_path, 'db_file_name': db_file_name,
                                             'drfit_db_name':drfit_db_name},
                                         dag=dag)

op_process_members = PythonOperator(task_id='process_members', 
                                    python_callable=get_transaction_data_transform,
                                    op_kwargs={'db_path': db_path, 'db_file_name': db_file_name,
                                             'drfit_db_name':drfit_db_name},
                                    dag=dag)

op_process_userlogs = PythonOperator(task_id='process_userlogs',
                                    python_callable=get_user_data_transform,
                                    op_kwargs={'db_path': db_path, 'db_file_name': db_file_name,
                                             'drfit_db_name':drfit_db_name},
                                    dag=dag)

op_merge = PythonOperator(task_id='merge_data',
                        python_callable=get_final_data_merge,
                        op_kwargs={'db_path': db_path, 'db_file_name': db_file_name,
                                             'drfit_db_name':drfit_db_name},
                        dag=dag)


op_process_data = PythonOperator(task_id='data_preparation', 
                            python_callable=get_data_prepared_for_modeling,
                            op_kwargs={'db_path': db_path,
                                       'db_file_name': db_file_name,
                                       'drfit_db_name':drfit_db_name,
                                       'date_columns':date_columns,
                                       'date_transformation':date_transformation
                                      },
                            dag=dag)

op_model_training_with_tuning = PythonOperator(task_id='Model_Training_hpTunning', 
                            python_callable=get_train_model_hptune,
                            op_kwargs={'db_path': db_path, 'db_file_name': db_file_name,'drfit_db_name':drfit_db_name},
                            dag=dag)


timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
send_email = EmailOperator( task_id='send_email', 
                                to='charliethomasctg@gmail.com', 
                                subject='Model Building Pipeline Execution Finished', 
                                html_content=f"The execution of Model Building pipeline is finished @ {timestamp}. Check Airflow Logs for more details or check MLFLOW for Final Model", 
                                dag=dag)




# Set the task sequence
op_reset_processes_flags.set_downstream(op_create_db)
op_create_db.set_downstream(op_load_data)
op_load_data.set_downstream([op_process_members,op_process_userlogs,op_process_transactions])
op_process_members.set_downstream(op_merge)
op_process_userlogs.set_downstream(op_merge)
op_process_transactions.set_downstream(op_merge)
op_merge.set_downstream(op_process_data)
op_process_data.set_downstream(op_model_training_with_tuning)
op_model_training_with_tuning.set_downstream(send_email)

